# Selección de variables


In [1]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go
import umap
from imblearn.over_sampling import RandomOverSampler

In [70]:
# Seteo jupyter notebook
pd.set_option('display.max_columns', None)

In [2]:
Data=pd.read_csv("../Datos/cleaned_05June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")


c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)

In [5]:
# Replace valores categoricos por númericos
for i in categorical:
    dic={}
    k=0
    for j in Data[i].unique():
        k=k+1
        dic[j]=k
    Data[i].replace(dic,inplace=True)

In [6]:
# Features - labels

features = Data.drop(columns=["Target"]).copy() 
labels = Data.loc[:,"Target"].copy()

# Generación de conjuntos de train y test
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33,random_state=50)


**Normalización de variables**

In [7]:
# Columns transformer
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMaxScaler',MinMaxScaler(),
         features.columns),
    ])

pipe = Pipeline(
    [("preprocesamiento", preprocessing_transformer)]
)

In [8]:
# Entrenamiento de pípeline
X_train=pd.DataFrame(pipe.fit_transform(X_train),columns=features.columns)

In [9]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

In [10]:
# ROS:
X_train, y_train=ros.fit_resample(X_train,y_train)

In [11]:
# Según el aporte de información con el criterio gini de un arbol de decisión
fs = DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=1)
fs.fit(X_train, y_train.astype(int))

DecisionTreeClassifier(max_depth=10, random_state=1)

In [12]:
var_num = 30
table = pd.DataFrame({'Tags': features.columns, 'Value': fs.feature_importances_})
table['Abs_Value'] = table['Value'].abs()
table = table[table['Abs_Value']>0].sort_values(by='Abs_Value', ascending=False, ignore_index=True)
tree_fs = table.loc[:var_num, 'Tags'].to_list()

In [13]:
# Plot
fig = go.Figure([go.Bar(x=table["Tags"], y=table["Abs_Value"])])

fig.update_layout(
    title_text="Importancia de variables",
)

fig.show()

 **Matriz de correlación**

In [14]:
correlation=(X_train.join(y_train)).corr()

In [15]:
aux_corr=pd.DataFrame(correlation["Target"].sort_values(ascending=False)).query("index!='Target'")

In [16]:
aux_corr

,Target
C-C,0.176361
Positivo,0.158486
D-F,0.149808
Principalidad,0.122645
Ciudad,0.115978
A-K,0.110405
lec_C-D,0.103330
num_comunicaciones,0.093075
B,0.087074
Frequency,0.081547


In [17]:
aux_corr.query("Target>0.1 and index!='Target'").index

Index(['C-C', 'Positivo', 'D-F', 'Principalidad', 'Ciudad', 'A-K', 'lec_C-D'], dtype='object')

In [18]:
# Corr
correlations = X_train.corr()

fig=px.imshow(correlations,
          labels=dict(x="", y="", color="Correlation"),
          x=X_train.columns,
          y=X_train.columns,
          zmin=-1,
          zmax=1,
          color_continuous_scale="Inferno")

fig.layout.height = 1000
fig.layout.width = 1000

fig.show()

In [19]:
def corrkill(dataframe, features, corr_cut=0.6):

    df = dataframe[features]
    dfcorr = pd.DataFrame(np.triu(df.corr()), columns=df.columns, index=df.columns)
    dfcorr = dfcorr.stack().reset_index()
    dfcorr.columns = ['Feat1','Feat2','Val']
    dfcorr = dfcorr[~dfcorr['Val'].isin([0,1])]
    dfcorr = dfcorr[dfcorr['Val'].abs()>corr_cut]
    
    return list(dfcorr['Feat2']), dfcorr

In [20]:
a,b=corrkill(X_train,X_train.columns,corr_cut=0.5)

In [21]:
b.sort_values('Val',ascending=False)

,Feat1,Feat2,Val
455,Frequency,Negativo,0.996173
161,lec_E-E,lec_D-E,0.895741
81,C,lec_B-B,0.827984
449,Frequency,A-K,0.650428
806,A-K,Negativo,0.641857
728,G-K,Negativo,0.638654
571,F-D,Positivo,0.631029
447,Frequency,G-K,0.621770
688,F-J,Positivo,0.585813
83,C,lec_D-E,0.570245


In [22]:
collinearity, table_cor=corrkill(features, tree_fs)
tag_select = [i for i in tree_fs if i not in collinearity]
tags_view_corr = features[tag_select].copy().reset_index(drop=True)

In [23]:
tag_select

['E-F',
 'D-F',
 'C-C',
 'A-K',
 'A-G',
 'num_comunicaciones',
 'Recency',
 'Positivo',
 'B',
 'Monto',
 'Periodicity',
 'Profesion',
 'lec_B-B',
 'Renta',
 'F-I',
 'Edad',
 'lec_A-A',
 'Comuna',
 'Sin Info',
 'Ciudad',
 'Duracion_Campaña',
 'Principalidad',
 'F-H',
 'lec_C-D']

In [24]:
# Corr
correlations = tags_view_corr.corr()

fig=px.imshow(correlations,
          labels=dict(x="", y="", color="Correlation"),
          x=tag_select,
          y=tag_select,
          zmin=-1,
          zmax=1,
          color_continuous_scale="Inferno")

fig.layout.height = 700
fig.layout.width = 700
fig.update_xaxes(
        tickangle = 45)

fig.show()